Name: Tan Wen Tao Bryan <br>
Admin No: P2214449 <br>
Class: DAAA/FT/2B/01<br>

-------------------------------------------------------------------------------------------------------------------------------

- Create docker container by pulling image for Docker Hub<br>
`docker pull python:3.8`<br>

- Run a Docker container with the name DLMODEL_Server_CA2 using the downloaded image for Python 3.8 <br>
`docker run -tid -v /var/run/docker.sock:/var/run/docker.sock --name DLMODEL_Server_CA2 python:3.8`<br>

In [1]:
# Importing the libraries
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.models import save_model
import matplotlib.pyplot as plt

2024-01-12 23:22:32.031150: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-12 23:22:32.069475: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-12 23:22:32.394356: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-12 23:22:32.395671: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-12 23:22:33.597353: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

### Export the models

In [2]:
# Load the models weights
veggie_cnn_31x31 = load_model('./DL_models/Final_31x31_Model.h5')
veggie_cnn_128x128 = load_model('./DL_models/Final_128x128_Model.h5')

# Save the models into img_classifier folder
save_model(veggie_cnn_31x31, "./img_classifier/veggie_cnn_31x31", save_format="tf")
save_model(veggie_cnn_128x128, "./img_classifier/veggie_cnn_128x128", save_format="tf")

INFO:tensorflow:Assets written to: ./img_classifier/veggie_cnn_31x31/assets


INFO:tensorflow:Assets written to: ./img_classifier/veggie_cnn_31x31/assets


INFO:tensorflow:Assets written to: ./img_classifier/veggie_cnn_128x128/assets


INFO:tensorflow:Assets written to: ./img_classifier/veggie_cnn_128x128/assets


### Deploying both models locally

- Serve the 2 models on tensorflow-serving and deploy it locally<br>
`docker run --name vegetable_cnn -p 8501:8501 -v "D:/ca2-daaa2b01-2214449-tanwentaobryan-img_classifier/img_classifier:/models/img_classifier" -t tensorflow/serving --model_config_file=/models/img_classifier/conf/models.config &`<br>

<b>models.config</b> <br>
```
model_config_list: {
  config: {
    name:  "veggie_cnn_31x31",
    base_path:  "/img_classifier/veggie_cnn_31x31",
    model_platform: "tensorflow",
    model_version_policy: {
        all: {}
    }
  },
  config: {
    name:  "veggie_cnn_128x128",
    base_path:  "/img_classifier/veggie_cnn_128x128",
    model_platform: "tensorflow",
    model_version_policy: {
        all: {}
    }
  }
}
```

Output: http://localhost:8501/v1/models/veggie_cnn_31x31<br>
```
{
    "model_version_status": [
        {
            "version": "2024011301",
            "state": "AVAILABLE",
            "status": {
                "error_code": "OK",
                "error_message": ""
            }
        }
    ]
}
```

Output: http://localhost:8501/v1/models/veggie_cnn_128x128<br>
```
{
    "model_version_status": [
        {
            "version": "2024011302",
            "state": "AVAILABLE",
            "status": {
                "error_code": "OK",
                "error_message": ""
            }
        }
    ]
}
```

### Connect both development container and vegetable_cnn to the same network

- Create a network<br>
`docker network create veggie_ml_network`<br>

- Connect vegetable_cnn to veggie_ml_network<br>
`docker network connect veggie_ml_network vegetable_cnn`<br>

- Connect DLMODEL_Server_CA2 to veggie_ml_network<br>
`docker network connect veggie_ml_network DLMODEL_Server_CA2`<br>

- Install ping program via bash<br>
`apt-get update`<br>
`apt-get install iputils-ping`<br>

- Check to see if the DLMODEL_Server_CA2 container is running and check the connection to vegetable_cnn <br>
`ping vegetable_cnn`


**Output:** <br>
`64 bytes from vegetable_cnn.veggie_ml_network (172.19.0.2): icmp_seq=21 ttl=64 time=0.042 ms`<br>
`64 bytes from vegetable_cnn.veggie_ml_network (172.19.0.2): icmp_seq=22 ttl=64 time=0.069 ms`

### test_docker_local.py 
- Testing was done to ensure that the 2 models are able to make predictions
- Checks if the local deployement of models work successfully
```
import pytest
import requests
import base64
import json
import numpy as np
from tensorflow.keras.preprocessing import image
import os

# Server URLs (test local deployment)
url_31x31 = "http://vegetable_cnn:8501/v1/models/veggie_cnn_31x31:predict"
url_128x128 = "http://vegetable_cnn:8501/v1/models/veggie_cnn_128x128:predict"

# Load test images from images folder
def load_image(img_size):
    local_path = os.path.join(os.getcwd(), 'tests/images')
    images_list = []
    for label in os.listdir(local_path):
        for filename in os.listdir(os.path.join(local_path,label)):
            # Load image of specified size and feature scale
            img = image.load_img(os.path.join(local_path,label, filename), color_mode='grayscale', target_size=(img_size, img_size))
            img = image.img_to_array(img)/255.0
            # Reshape image to (1, img_size, img_size, 1)
            img = img.reshape(1, img_size, img_size, 1)
            images_list.append(img)
    return images_list

# Predict using test images
def make_prediction(instances, url):
    # Send POST API request to server
    data = json.dumps({"signature_name": "serving_default", "instances": instances.tolist()})
    headers = {"content-type": "application/json"}
    json_response = requests.post(url, data=data, headers=headers)
    # Parse response
    predictions = json.loads(json_response.text)['predictions']
    return predictions

# Test prediction for 31x31 model
def test_prediction_31():
    data = load_image(31)
    for img in data:
        predictions = make_prediction(img, url_31x31)
        # Check if prediction is a list
        assert isinstance(predictions, list)
        # Check if prediction is a list of length 1
        assert len(predictions) == 1
        # Check if each prediction is a float
        assert isinstance(predictions[0][0], float)

# Test prediction for 128x128 model
def test_prediction_128():
    data = load_image(128)
    for img in data:
        predictions = make_prediction(img, url_128x128)
        # Check if prediction is a list
        assert isinstance(predictions, list)
        # Check if prediction is a list of length 1
        assert len(predictions) == 1
        # Check if prediction is a float
        assert isinstance(predictions[0][0], float)
```

### Deploying both models remotely
<b>Dockerfile</b> <br>
```
FROM tensorflow/serving
COPY / /
ENV MODEL_CONF=/img_classifier/models.config MODEL_BASE_PATH=/
EXPOSE 8500
EXPOSE 8501
RUN echo '#!/bin/bash \n\n\
tensorflow_model_server \
--rest_api_port=$PORT \
--model_config_file=${MODEL_CONF} \
"$@"' > /usr/bin/tf_serving_entrypoint.sh \
&& chmod +x /usr/bin/tf_serving_entrypoint.sh
```
Code Meaning:
- `FROM tensorflow/serving` - Initializes a new build stage and sets the Base Image (tensorflow/serving) for subsequent instructions.
- `COPY / /` - Copies new files or directories from source `root` and adds them to the file system of the container at the path `root`.
- `ENV MODEL_CONF=/img_classifier/models.config` - Sets environment variable to the specified config file
- `EXPOSE 8500 EXPOSE 8501` - Exposes specified ports to inform Docker that the container listens on the specified network ports at runtime
- `RUN echo '#!/bin/bash \n\n
tensorflow_model_server 
--rest_api_port=$PORT 
--model_config_file=${MODEL_CONF} 
"$@"' > /usr/bin/tf_serving_entrypoint.sh 
&& chmod +x /usr/bin/tf_serving_entrypoint.sh` - Executes any commands in a new layer on top of the current image and commit the results

- Lastly, the models are deployed on render as a web service via Docker<br>

Output: https://veggietales-cnn.onrender.com/v1/models/veggie_cnn_31x31<br>
```
{
 "model_version_status": [
  {
   "version": "202401",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": ""
   }
  }
 ]
}
```


Output: https://veggietales-cnn.onrender.com/v1/models/veggie_cnn_128x128<br>
```
{
 "model_version_status": [
  {
   "version": "202402",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": ""
   }
  }
 ]
}
```

### test_docker_global.py 
- Testing was done to ensure that the 2 models are able to make predictions
- Checks if the remote deployement of models on render work successfully
```
import pytest
import requests
import base64
import json
import numpy as np
from tensorflow.keras.preprocessing import image
import os

# Server URLs (test rempte deployment)
url_31x31 = "https://veggietales-cnn.onrender.com/v1/models/veggie_cnn_31x31:predict"
url_128x128 = "https://veggietales-cnn.onrender.com/v1/models/veggie_cnn_128x128:predict"

# Load test images from images folder
def load_image(img_size):
    local_path = os.path.join(os.getcwd(), 'tests/images')
    images_list = []
    for label in os.listdir(local_path):
        for filename in os.listdir(os.path.join(local_path,label)):
            # Load image of specified size and feature scale
            img = image.load_img(os.path.join(local_path,label, filename), color_mode='grayscale', target_size=(img_size, img_size))
            img = image.img_to_array(img)/255.0
            # Reshape image to (1, img_size, img_size, 1)
            img = img.reshape(1, img_size, img_size, 1)
            images_list.append(img)
    return images_list

# Predict using test images
def make_prediction(instances, url):
    # Send POST API request to server
    data = json.dumps({"signature_name": "serving_default", "instances": instances.tolist()})
    headers = {"content-type": "application/json"}
    json_response = requests.post(url, data=data, headers=headers)
    # Parse response
    predictions = json.loads(json_response.text)['predictions']
    return predictions

# Test prediction for 31x31 model
def test_prediction_31():
    data = load_image(31)
    for img in data:
        predictions = make_prediction(img, url_31x31)
        # Check if prediction is a list
        assert isinstance(predictions, list)
        # Check if prediction is a list of length 1
        assert len(predictions) == 1
        # Check if each prediction is a float
        assert isinstance(predictions[0][0], float)

# Test prediction for 128x128 model
def test_prediction_128():
    data = load_image(128)
    for img in data:
        predictions = make_prediction(img, url_128x128)
        # Check if prediction is a list
        assert isinstance(predictions, list)
        # Check if prediction is a list of length 1
        assert len(predictions) == 1
        # Check if prediction is a float
        assert isinstance(predictions[0][0], float)
```

### Output of both tests - both tests pass
```
tests/test_docker_global.py ..                                                                                                                                                 [ 50%]
tests/test_docker_local.py ..                                                                                                                                                  [100%]

====================================================================== 4 passed in 4.10s ====================================================================
```

### CI/CD - Model Testing & Deployment

```
stages:
  - test
  - deploy
  - notification

pytest:
  stage: test
  before_script:
    - pwd
  image: python:3.8
  script:
    - pip install -r requirements.txt
    # Skip local deployment test because docker does not store the local files used to run the test in the container environment
    - python -m pytest -k "not tests/test_docker_local.py" --junitxml=junit.xml
  artifacts:
    reports:
      junit: junit.xml

deployment:
  stage: deploy
  script:
    - curl https://veggietales-cnn.onrender.com/v1/models/veggie_cnn_31x31
    - curl https://veggietales-cnn.onrender.com/v1/models/veggie_cnn_128x128
  only:
    - main

notification:
  stage: notification
  script:
    # Notifies team channel if pipeline succeed or failed on discord channel through discord webhooks
    - 'if [ "$CI_PIPELINE_STATUS" == "success" ]; then curl -X POST -H "Content-type: application/json" --data "{\"content\":\"Pipeline Succeeded: $CI_COMMIT_REF_NAME - $CI_COMMIT_TITLE\"}" https://discord.com/api/webhooks/1200457121326710914/8NBM-ImDrvrtSmNB-Ynay07tBz9H5NvLC64gJSF9W1Vu5DluE-UrulhCuCu28gYy08Hy; fi'
    - 'if [ "$CI_PIPELINE_STATUS" == "failed" ]; then curl -X POST -H "Content-type: application/json" --data "{\"content\":\"Pipeline Failed: $CI_COMMIT_REF_NAME - $CI_COMMIT_TITLE\"}" https://discord.com/api/webhooks/1200457121326710914/8NBM-ImDrvrtSmNB-Ynay07tBz9H5NvLC64gJSF9W1Vu5DluE-UrulhCuCu28gYy08Hy; fi'
```